In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Set parameters
latent_dim = 100
img_shape = (64, 64, 3)  # Adjusted size
epochs = 100
batch_size = 64

# Build the generator
def build_generator():
    model = keras.Sequential([
        layers.Dense(128 * 16 * 16, activation="relu", input_dim=latent_dim),
        layers.Reshape((16, 16, 128)),
        layers.UpSampling2D(),
        layers.Conv2D(128, kernel_size=3, padding='same'),
        layers.Activation('relu'),
        layers.UpSampling2D(),
        layers.Conv2D(64, kernel_size=3, padding='same'),
        layers.Activation('relu'),
        layers.Conv2D(3, kernel_size=3, padding='same', activation='tanh')  # Change to output (64, 64, 3)
    ])
    return model

# Build the discriminator
def build_discriminator():
    model = keras.Sequential([
        layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(64, 64, 3)),  # Updated input shape
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size=3, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Compile GAN
generator = build_generator()
discriminator = build_discriminator()

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create GAN model
z = layers.Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = keras.Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer='adam')

# Load your dataset
(x_train, _), (_, _) = keras.datasets.cifar10.load_data()  # Sample dataset
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize images
x_train = x_train[np.random.choice(x_train.shape[0], size=5000, replace=False)]  # Example for using a subset

# Assuming x_train is defined and properly loaded
# Training GAN
for epoch in range(epochs):
    # Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    # Resize real_imgs to match discriminator's input shape
    real_imgs = tf.image.resize(real_imgs, (64, 64))

    # Generate a batch of new images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))

    # Average the losses and accuracies
    d_loss = (d_loss_real[0] + d_loss_fake[0]) / 2  # Average loss
    d_acc = (d_loss_real[1] + d_loss_fake[1]) / 2  # Average accuracy

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print the progress
    if epoch % 1000 == 0:
        # print(f"{epoch} [D loss: {d_loss:.4f}, acc.: {100 * d_acc:.2f}%] [G loss: {g_loss:.4f}]")

        # Save generated images
        generated_imgs = generator.predict(noise)
        generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]
        plt.figure(figsize=(10, 10))
        for i in range(9):
            plt.subplot(3, 3, i + 1)
            plt.imshow(generated_imgs[i])
            plt.axis('off')
        plt.show()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
